In [1]:
import numpy as np
import pandas as pd

df_airline_data = pd.read_csv('86206597_102016_1233_airline_delay_causes.csv')
df_airports = pd.read_csv('airports.csv')
df_merge = pd.merge(df_airline_data, df_airports[['iata','lat','long']], how='left', left_on='airport', right_on='iata')
df_merge.carrier_name = df_merge.carrier_name.apply(lambda x: x.replace(' Inc.',''))
df_merge.carrier_name = df_merge.carrier_name.apply(lambda x: x.replace(' Co.',''))
df_merge.carrier_name.unique()
del df_merge['iata']
del df_merge['Unnamed: 21']
df_merge.head()

year   month carrier       carrier_name airport  \
0  2003       6      AA  American Airlines     ABQ   
1  2003       6      AA  American Airlines     ANC   
2  2003       6      AA  American Airlines     ATL   
3  2003       6      AA  American Airlines     AUS   
4  2003       6      AA  American Airlines     BDL   

                                        airport_name  arr_flights  arr_del15  \
0  Albuquerque, NM: Albuquerque International Sun...        307.0       56.0   
1  Anchorage, AK: Ted Stevens Anchorage Internati...         90.0       27.0   
2  Atlanta, GA: Hartsfield-Jackson Atlanta Intern...        752.0      186.0   
3       Austin, TX: Austin - Bergstrom International        842.0      174.0   
4                Hartford, CT: Bradley International        383.0       55.0   

   carrier_ct   weather_ct     ...      arr_cancelled  arr_diverted  \
0       14.68        10.79     ...                1.0           1.0   
1        7.09         2.00     ...                0.0           0.0   
2       33.99        27.82     ...                5.0           0.0   
3       60.24        20.54     ...                9.0           1.0   
4       14.90         8.91     ...                0.0           0.0   

    arr_delay   carrier_delay  weather_delay  nas_delay  security_delay  \
0      2530.0           510.0          621.0      676.0            25.0   
1      1390.0           271.0           83.0      581.0             0.0   
2      8314.0          1367.0         1722.0     3817.0           139.0   
3      8344.0          3040.0         1032.0     1835.0           115.0   
4      3137.0           815.0          574.0      555.0             0.0   

   late_aircraft_delay        lat        long  
0                698.0  35.040222 -106.609194  
1                455.0  61.174320 -149.996186  
2               1269.0  33.640444  -84.426944  
3               2322.0  30.194533  -97.669872  
4               1193.0  41.938874  -72.683228  

[5 rows x 23 columns]

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

wiki = "https://en.wikipedia.org/wiki/List_of_hub_airports#United_States"
r = requests.get(wiki)
data = r.text
soup = BeautifulSoup(data, 'html5lib')
hub_data = soup.findAll("table", {"class" : "wikitable"})
hub_data = hub_data[1]

In [3]:
records = dict([('country',[]),('state',[]),('airport',[]),('carrier_name',[])])
for row in hub_data.findAll('tr'):
    try:
        col = row.findAll('td')
        if col == []:
            continue
        airline = col[3]
        airline = str(airline.contents[0])
        airline = re.sub(r'\([^)]*\)','',airline).strip()
        airline = re.sub(r'>','',airline)
        airport = col[2].string.strip()
        airport = re.split('\(',airport)[1].replace(')','')
        records['airport'].append(airport)
        records['carrier_name'].append(airline)
        records['country'].append(col[0].string.strip())
        records['state'].append(col[1].string.strip())
    except:
        continue
df_hub = pd.DataFrame.from_dict(records)

In [4]:
df_hub.head()

airport        carrier_name        country    state
0     ANC     Alaska Airlines  United States   Alaska
1     ANC  Northwest Airlines  United States   Alaska
2     ANC              PenAir  United States   Alaska
3     PHX   American Airlines  United States  Arizona
4     PHX          US Airways  United States  Arizona

In [5]:
df_merge = pd.merge(df_merge, df_hub, how='left', on=['airport','carrier_name'])
df_merge['is_hub'] = (df_merge['country'].notnull()).astype('int')
del df_merge['country']
del df_merge['state']
df_merge.head(10)

year   month carrier       carrier_name airport  \
0  2003       6      AA  American Airlines     ABQ   
1  2003       6      AA  American Airlines     ANC   
2  2003       6      AA  American Airlines     ATL   
3  2003       6      AA  American Airlines     AUS   
4  2003       6      AA  American Airlines     BDL   
5  2003       6      AA  American Airlines     BHM   
6  2003       6      AA  American Airlines     BNA   
7  2003       6      AA  American Airlines     BOS   
8  2003       6      AA  American Airlines     BUR   
9  2003       6      AA  American Airlines     BWI   

                                        airport_name  arr_flights  arr_del15  \
0  Albuquerque, NM: Albuquerque International Sun...        307.0       56.0   
1  Anchorage, AK: Ted Stevens Anchorage Internati...         90.0       27.0   
2  Atlanta, GA: Hartsfield-Jackson Atlanta Intern...        752.0      186.0   
3       Austin, TX: Austin - Bergstrom International        842.0      174.0   
4                Hartford, CT: Bradley International        383.0       55.0   
5  Birmingham, AL: Birmingham-Shuttlesworth Inter...         89.0       12.0   
6             Nashville, TN: Nashville International        445.0       82.0   
7                    Boston, MA: Logan International       1266.0      225.0   
8                              Burbank, CA: Bob Hope        119.0       27.0   
9  Baltimore, MD: Baltimore/Washington Internatio...        593.0      101.0   

   carrier_ct   weather_ct   ...    arr_diverted   arr_delay   carrier_delay  \
0       14.68        10.79   ...             1.0      2530.0           510.0   
1        7.09         2.00   ...             0.0      1390.0           271.0   
2       33.99        27.82   ...             0.0      8314.0          1367.0   
3       60.24        20.54   ...             1.0      8344.0          3040.0   
4       14.90         8.91   ...             0.0      3137.0           815.0   
5        2.79         2.19   ...             0.0       673.0            94.0   
6       25.44        11.98   ...             0.0      4663.0          1217.0   
7       69.43        23.66   ...             0.0     12139.0          4201.0   
8        7.49         4.65   ...             0.0      1187.0           326.0   
9       17.56        20.49   ...             1.0      5698.0          1058.0   

   weather_delay  nas_delay  security_delay  late_aircraft_delay        lat  \
0          621.0      676.0            25.0                698.0  35.040222   
1           83.0      581.0             0.0                455.0  61.174320   
2         1722.0     3817.0           139.0               1269.0  33.640444   
3         1032.0     1835.0           115.0               2322.0  30.194533   
4          574.0      555.0             0.0               1193.0  41.938874   
5           90.0       50.0            35.0                404.0  33.562943   
6          913.0      800.0             1.0               1732.0  36.124477   
7         1783.0     3067.0            45.0               3043.0  42.364348   
8          191.0      285.0             0.0                385.0  34.200619   
9         1332.0     1708.0             0.0               1600.0  39.175402   

         long  is_hub  
0 -106.609194       0  
1 -149.996186       0  
2  -84.426944       0  
3  -97.669872       0  
4  -72.683228       0  
5  -86.753550       0  
6  -86.678182       1  
7  -71.005179       0  
8 -118.358497       0  
9  -76.668198       0  

[10 rows x 24 columns]

In [6]:
# Look only at hub data -> Remove all rows with 'is_hub' equal to zero to reduce dataset
df_merge = df_merge[df_merge.is_hub != 0]
df_merge.head()

year   month carrier       carrier_name airport  \
6   2003       6      AA  American Airlines     BNA   
11  2003       6      AA  American Airlines     CLT   
15  2003       6      AA  American Airlines     DCA   
17  2003       6      AA  American Airlines     DFW   
35  2003       6      AA  American Airlines     JFK   

                                         airport_name  arr_flights  arr_del15  \
6              Nashville, TN: Nashville International        445.0       82.0   
11     Charlotte, NC: Charlotte Douglas International        203.0       48.0   
15  Washington, DC: Ronald Reagan Washington National        947.0      147.0   
17  Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...      12276.0     2144.0   
35        New York, NY: John F. Kennedy International       1147.0      212.0   

    carrier_ct   weather_ct   ...    arr_diverted   arr_delay   carrier_delay  \
6        25.44        11.98   ...             0.0      4663.0          1217.0   
11       16.57         9.72   ...             0.0      3048.0           968.0   
15       42.88        24.89   ...             2.0      7868.0          2048.0   
17      392.85        66.72   ...            66.0    117195.0         31999.0   
35       60.92        14.66   ...             4.0     10785.0          3496.0   

    weather_delay  nas_delay  security_delay  late_aircraft_delay        lat  \
6           913.0      800.0             1.0               1732.0  36.124477   
11          742.0      558.0           127.0                653.0  35.214011   
15         1484.0     2767.0             3.0               1566.0  38.852083   
17         4787.0    43864.0           459.0              36086.0  32.895951   
35         1157.0     3596.0           203.0               2333.0  40.639751   

         long  is_hub  
6  -86.678182       1  
11 -80.943126       1  
15 -77.037722       1  
17 -97.037200       1  
35 -73.778926       1  

[5 rows x 24 columns]

In [27]:
df_merge.to_csv('airline_delay_loc.csv')

In [23]:
pd.unique(df_merge.carrier_name)
pd.unique(df_merge[df_merge.carrier == 'NW'].airport_name)

array(['Anchorage, AK: Ted Stevens Anchorage International',
       'Detroit, MI: Detroit Metro Wayne County',
       'Memphis, TN: Memphis International',
       'Minneapolis, MN: Minneapolis-St Paul International'], dtype=object)